### 1. 라이브러리 추가

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import LabelEncoder

### 2. 데이터 정제

In [3]:
featureData = pd.read_csv("OT_F_DATA_V4.csv")

In [4]:
featureData['PROD_NM'].unique()

array(['헬로LTE 표준', '슬림 2GB', '슬림 1GB', '슬림 250MB', 'The 착한 데이터 10GB',
       'The 착한 데이터 6GB', 'The 착한 데이터 2GB', 'The 착한 데이터 2GB(new)',
       'The 착한 데이터 1GB', 'The 착한 데이터 300MB', '헬로LTE42(안심차단)',
       '헬로LTE34(안심차단)', '헬로LTE29', '헬로LTE17', '헬로LTE 청소년윙 19(new)'],
      dtype=object)

In [5]:
featureData.dtypes

USE_YYMM                int64
CTRT_ID                object
PROD_ID                object
PROD_NM                object
USE_VOICE               int64
USE_OVER_VOICE_AMT    float64
USE_SMS                 int64
USE_OVER_SMS_AMT        int64
USE_DATA                int64
USE_OVER_DATA_AMT     float64
PROD_AMT                int64
PROD_VOICE              int64
PROD_SMS                int64
PROD_DATA               int64
DATA_LIMIT_YN           int64
CUST_ID                object
ACT_DTTM                int64
ORDER_TP                int64
PROMOTION_NM           object
CUST_TP                object
SEX_TP                 object
BIRTH_DT                int64
AGE                     int64
SEX_NUM                 int64
TOTAL_AMT               int64
dtype: object

In [6]:
le_prod_id = LabelEncoder()

In [7]:
featureData["LE_PROD_NM"] = le_prod_id.fit_transform(featureData.PROD_NM)

In [8]:
featureData.head()
featureData['LE_PROD_NM'].unique()

array([10,  8,  6,  7,  0,  5,  2,  3,  1,  4, 14, 13, 12, 11,  9],
      dtype=int64)

In [9]:
features = ['AGE',
            'SEX_NUM',
            'TOTAL_AMT',
            'PROD_AMT',
            'USE_DATA'
            ]
label = ['LE_PROD_NM']

In [10]:
de_norm = featureData[features].max()

In [11]:
de_norm[0]

107

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
trainingData_features,testData_features,trainingData_label,testData_label,trainingData_all,testData_all = \
train_test_split(featureData[features],featureData[label],featureData, test_size=0.2, random_state=1)

In [14]:
from tensorflow.keras.utils import to_categorical

In [15]:
trainingData_labels_one = to_categorical(trainingData_label)
testData_labels_one = to_categorical(testData_label)

In [16]:
def norm(x):
    return (x/x.max())

In [17]:
trainingData_features_normed = norm(trainingData_features)
testData_features_normed = norm(testData_features)

### 3. 모델 생성

In [18]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import neighbors

In [19]:
inputDim = trainingData_features.loc[0,:].shape
inputDim

(5,)

In [20]:
outputShape = len(featureData['LE_PROD_NM'].unique())
outputShape

15

In [28]:
model_method_svm = svm.SVC(random_state=1)
# model_method_dt = tree.DecisionTreeClassifier(random_state =1)    #DecisionTreeRegressor(random_state =1)
# model_method_kn = neighbors.KNeighborsClassifier(n_neighbors=10)
# model_method_rf = ensemble.RandomForestClassifier(random_state =1, n_estimators= 10)

In [29]:
model = Sequential()
model.add(Dense(units=8, activation="relu",input_shape=inputDim))
model.add(Dense(units=8, activation="relu"))
model.add(Dense(units=8, activation="relu"))
model.add(Dense(units=outputShape, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 48        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 15)                135       
Total params: 327
Trainable params: 327
Non-trainable params: 0
_________________________________________________________________


### 4. 모델 컴파일

In [30]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### 5. 모델 학습

In [ ]:
epochNo = 5

In [33]:
model.fit(x=trainingData_features_normed,y=trainingData_labels_one, epochs=epochNo,
          batch_size = 32,
         validation_data=(testData_features_normed,testData_labels_one))

Train on 1289932 samples, validate on 322484 samples
Epoch 1/5
1289932/1289932 [==============================] - 69s 54us/sample - loss: 0.2549 - accuracy: 0.8908 - val_loss: 0.1345 - val_accuracy: 0.9217
Epoch 2/5
1289932/1289932 [==============================] - 75s 58us/sample - loss: 0.1368 - accuracy: 0.9280 - val_loss: 0.1399 - val_accuracy: 0.9254
Epoch 3/5
1289932/1289932 [==============================] - 74s 57us/sample - loss: 0.1251 - accuracy: 0.9349 - val_loss: 0.1183 - val_accuracy: 0.9387
Epoch 4/5
1289932/1289932 [==============================] - 72s 56us/sample - loss: 0.1150 - accuracy: 0.9406 - val_loss: 0.1428 - val_accuracy: 0.9260
Epoch 5/5
1289932/1289932 [==============================] - 72s 56us/sample - loss: 0.1080 - accuracy: 0.9440 - val_loss: 0.1763 - val_accuracy: 0.9183


### 모델 불러오기

In [22]:
from tensorflow.keras.models import model_from_json

In [23]:
json_file = open("c:/Users/user/model_recomand_prod.json","r")
loaded_model_json = json_file.read()
json_file.close()

In [24]:
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("c:/Users/user/model_recomand_weight.h5")

### 추론

,AGE,SEX_NUM,TOTAL_AMT,PROD_AMT,USE_DATA
1538448,0.11215,1.0,0.016022,0.346084,0.0


In [46]:
np.argmax(model.predict(testData_features_normed.head(1)))

9

In [130]:
le_prod_id.inverse_transform([9])

array(['헬로LTE 청소년윙 19(new)'], dtype=object)

In [162]:
def denorm(x):
    for i in range(0,len(x.columns)):
        print(x.iloc[0][i] * de_norm[i])
    
    
    return x

In [171]:
def denorm(x):
    for i in range(0,len(x.columns)):
        print( x.columns.array[i] + ' : ' +  str(x.iloc[0][i] * de_norm[i]))
        
        
    return x
    
    

In [245]:
testNorm = denorm(testData_features_normed.head(1))
testNorm
# ;
# testNorm = testData_features_normed.head(1)


AGE : 10.0
SEX_NUM : 1.0
TOTAL_AMT : 30000.0
PROD_AMT : 20000.0
USE_DATA : 4866.894353369764


,AGE,SEX_NUM,TOTAL_AMT,PROD_AMT,USE_DATA
1538448,0.093458,0.5,0.006578,0.364299,0.009107


In [210]:
testNorm.iloc[0][0]

0.09345794392523364

In [211]:
testNorm.columns.array[0]

'AGE'

In [212]:
de_norm

AGE              107
SEX_NUM            2
TOTAL_AMT    4560977
PROD_AMT       54900
USE_DATA      534385
dtype: int64

In [239]:
testNormT= trainingData_features.iloc[:1]

In [240]:
testNormT

,AGE,SEX_NUM,TOTAL_AMT,PROD_AMT,USE_DATA
1065970,63,2,28900,28900,127


In [228]:
# testNormT = pd.DataFrame(testNormT)

In [241]:
testNormT['AGE'] = 10/ de_norm[0]
testNormT['SEX_NUM'] = 1 / de_norm[1]
testNormT['TOTAL_AMT'] = 30000 / de_norm[2]
testNormT['PROD_AMT'] = 20000 / de_norm[3]
testNormT['USE_DATA'] = 500 / de_norm[3]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [242]:
testNormT

,AGE,SEX_NUM,TOTAL_AMT,PROD_AMT,USE_DATA
1065970,0.093458,0.5,0.006578,0.364299,0.009107


In [243]:
predict = np.argmax(model.predict(testNormT))

In [244]:
le_prod_id.inverse_transform([predict])

array(['헬로LTE 청소년윙 19(new)'], dtype=object)

### 모델 저장

In [58]:
model_json = model.to_json()
with open("c:/Users/user/model_recomand_prod.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("c:/Users/user/model_recomand_weight.h5")

In [56]:
pwd

'C:\\Users\\User\\python'